# Lecture 19 – Exploratory Data Analysis: Voter Targeting in Pennsylvania

## Data 6, Summer 2021

In [ ]:
# Importing all of our tools
import numpy as np
from datascience import *
from math import *

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import plotly.express as px
Table.interactive_plots()

Run the next cell to import all of our datasets, obtained from https://www.electionreturns.pa.gov/ and https://www.truckads.com/. 

In [ ]:
pa_2020 = Table().read_table("data/2020_presidential.CSV")
pa_2020.show(5)

## Step 1: Data Cleaning

To make this easier, let's cut it down to the columns we want. (For this section, don't re-run cells! You'll get an error.)

In [ ]:
cols_of_interest = ["County Name", "Party Name", "Candidate Name", "Votes", "Election Day Votes", "Mail Votes", "Provisional Votes"]

pa_2020 = pa_2020.select(cols_of_interest)
pa_2020

In [ ]:
# What was the voter turn out (# of voters who voted) in 2020? Try calculating it.
total_votes = sum(pa_2020.column("Votes"))
total_votes

In [ ]:
# What's the problem? Let's check the data types.
pa_2020.column("Votes").item(0)

In [ ]:
# It looks like we need to do some data cleaning. How do we convert this into a number we can work with?
adamsctybiden = pa_2020.column("Votes").item(0)

# strings are a specific object with methods associated
# remember we can "cast" a datatype to a int
int(adamsctybiden.replace(",", ""))

In [ ]:
# Now, how do we do this to all of the numbers in the dataset? We need a function.

def vote_to_int(votes):
    """Given a string of votes, convert to an integer"""
    return int(votes.replace(",", "")) 

vote_to_int(adamsctybiden)

In [ ]:
# To do a lot of conversions, we can apply this to the data in the table!

votes_as_int = pa_2020.apply(vote_to_int, "Votes")

sum(votes_as_int) # Now we can do math!

In [ ]:
# NOTE: We can do the same thing with a for loop:
looped_ver = np.array([])

for vote in pa_2020.column("Votes"):
    looped_ver = np.append(looped_ver, vote_to_int(vote))

looped_ver

In [ ]:
# Now, we need to convert all string data into integer data
cols_to_convert = ["Votes", "Election Day Votes", "Mail Votes", "Provisional Votes"]

for col in cols_to_convert:
    votes_as_ints = pa_2020.apply(vote_to_int, col)
    pa_2020 = pa_2020.with_column(col, votes_as_ints)
    
pa_2020

In [ ]:
# Checking data types again, just in case
type(pa_2020.column("Mail Votes").item(0))

In [ ]:
# Looks good! Now, as someone interested in media, let's add in some info about TV media markets
pa_media_markets = Table().read_table("data/media_markets.csv")
pa_media_markets.show(5)

In [ ]:
# I want this information attached to pa_2020
# We'll learn this tomorrow: we can "join" multiple tables by a specific column
# One issue: the data needs to be the same across tables to work,
# so the upper case vs capitalized counties will give us an error

mm_counties = pa_media_markets.column("County")
cleaned_counties = make_array()

for county in mm_counties:
    clean_county = county.upper() # Converting counties into upper case to make this work
    cleaned_counties = np.append(cleaned_counties, clean_county)

cleaned_counties

In [ ]:
# Cleaning up the media markets table with the new names
pa_media_markets = pa_media_markets.with_column("County", cleaned_counties)
pa_media_markets

In [ ]:
# Finally, combining all of our data with a join
pa_2020_mm = pa_2020.join("County Name", pa_media_markets, "County")
pa_2020_mm

# Step 2: Exploratory Data Analysis

**Our questions for this analysis:**
- **How did each party vote?** Remember that the GOP was very against mail-in-ballots in 2020. 

- **What media markets supported each party in 2020?** Pennsylvania is going through a demographic and economic shift, so this may have insights for 2022 or 2024.


Let's learn more about where voters voted and how they voted. We're going to use `group` and `pivot` for this.

Recall: `tbl.group("col", func)` If func is not specified, by default finds the count of each unique value in "col". Otherwise, applies func to the grouped values in every other column. 

`tbl.pivot("col", "row", "vals", func)` cross-classifies a dataset, making all the unique values in 1 column the new rows and all the unique values in the other column the new column labels. Then, it puts the values of "vals", with the function applied to each group, in the corresponding cells.

For example:
http://data8.org/interactive_table_functions/ 

In [ ]:
# Now we can finally begin a quick analysis using interesting table methods
# First, we can use a group to quickly quantify the # of counties in each media market

fixed_counts = pa_2020_mm.group("Media Market").column("count") / 3 
# The dataset has 3 entries per Media Market, so divide by 3 to fix that

pa_2020_mm.group("Media Market").with_column("count", fixed_counts)

In [ ]:
# How many voters are in each media market? 
# Notice that we need to clean the resultant table a bit by dropping extra columns

pa_2020_mm.group("Media Market", sum).drop(1, 2, 3)

In [ ]:
sum(pa_2020_mm.where("Media Market", "Erie").column("Votes"))

In [ ]:
# We can also use a double group to quantify by both media market and party
pa_2020_mm.group(["Media Market", "Party Name"], sum).drop(2, 3)

In [ ]:
# In general, how did each party vote? (Election Day, Mail Votes, Provisional)
# Let's figure it out. 

party_by_votes = pa_2020_mm.group("Party Name", sum).drop([1, 2, 7])
party_by_votes

In [ ]:
# Let's convert all of the columns into a proportion using array arithmetic; just run this cell
# This makes it a bit easier to compare across party by controlling by # of votes

def vote_prop(col_str):
    return party_by_votes.column(col_str) / party_by_votes.column("Votes sum")

party_vote_props = party_by_votes.select("Party Name").with_columns("Election Day Votes", vote_prop("Election Day Votes sum"),
                                                                   "Mail Votes", vote_prop("Mail Votes sum"),
                                                                   "Provisional Votes", vote_prop("Provisional Votes sum"))

# Bar chart to see the breakdown! 
party_vote_props.barh(0)

In [ ]:
# Reminder of the table setup
pa_2020_mm.show(5)

Now let's try "cross classifying"; this is similar to a 2 column group, but let's focus on a specific question:

**How were the votes broken down by Media Market and party?**

Or, in other words, what media markets provided most of the raw votes come from for each party? This is useful information because electoral college votes are based on state totals (i.e. it doesn't matter if Johnstown Altoona is very Republican if there aren't many voters there). 

In [ ]:
# Pivots the table so we have parties as column labels (arg 1), media markets as rows (arg 2), and
# vote totals as cell values (arg 3); we aggregate vote totals with sum (arg 4)
market_vs_party = pa_2020_mm.pivot("Party Name", "Media Market", "Votes", sum)
market_vs_party

In [ ]:
# Let's look at the data with a bar chart
# What does this tell us?
market_vs_party.barh(0)

In [ ]:
# Now, what if I wanted to figure out the preferred party for each media market?
# Just run this cell; it's a lot of wordy array arithmetic: we're converting each party to proportions

market_vs_party.column("Democratic") / (market_vs_party.column("Democratic") + market_vs_party.column("Libertarian") + market_vs_party.column("Republican"))

mvp_props = market_vs_party.select("Media Market").with_columns("Democratic", market_vs_party.column("Democratic") / (market_vs_party.column("Democratic") + market_vs_party.column("Libertarian") + market_vs_party.column("Republican")),
                                                                "Libertarian", market_vs_party.column("Libertarian") / (market_vs_party.column("Democratic") + market_vs_party.column("Libertarian") + market_vs_party.column("Republican")),
                                                                "Republican", market_vs_party.column("Republican") / (market_vs_party.column("Democratic") + market_vs_party.column("Libertarian") + market_vs_party.column("Republican")))

# Now that we've controlled for number of votes in media market, this shows us who "won" each media market
mvp_props.barh(0)

## Additional Analysis: Targeting by Market and Understanding What Happened in 2020

If you're interested, check out **[this notebook](./pa_analysis_full.ipynb)** by Ian that gives us more insight into the state by looking at more elections data.

You'll notice that the linked notebook uses a different syntax -- that's because it uses `pandas` instead of the `datascience` library that we're used to! 

The `datascience` library is great because it's simple and easier to learn, but has a lot of limitations and requires a lot of work to do just this level of cleaning and EDA. You can learn in further data science classes about `pandas`, which is a lot more powerful, concise, and lets us do even more interesting analysis with data tables.